Ejercicio: Más métricas derivadas de matrices de confusión

En este ejercicio aprenderemos sobre diferentes métricas, usándolas para explicar los resultados obtenidos del modelo de clasificación binaria que construimos en la unidad anterior.

Visualización de datos
Usaremos el conjunto de datos con diferentes clases de objetos encontrados en la montaña una vez más:

In [ ]:
import pandas
import numpy
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

#Let's have a look at the data
dataset

Recuerde que para usar el conjunto de datos anterior para la clasificación binaria, debemos agregar otra columna al conjunto de datos y configurarlo en Verdadero donde la etiqueta original es excursionista y Falso donde no lo es.

Luego agreguemos esa etiqueta, dividamos el conjunto de datos y entrenemos el modelo nuevamente

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Add a new label with true/false values to our dataset
dataset["is_hiker"] = dataset.label == "hiker"

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=1, shuffle=True)

# define a random forest model
model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False)

# Define which features are to be used 
features = ["size", "roughness", "motion"]

# Train the model using the binary label
model.fit(train[features], train.is_hiker)

print("Model trained!")

Ahora podemos usar este modelo para predecir si los objetos en la nieve son excursionistas o no.

In [ ]:
# sklearn has a very convenient utility to build confusion matrices
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Calculate the model's accuracy on the TEST set
actual = test.is_hiker
predictions = model.predict(test[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Create the list of unique labels in the test set, to use in our plot
# I.e., ['True', 'False',]
unique_targets = sorted(list(test["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

In [ ]:
# Let's also calculate some values that will be used throughout this exercise
# We already have actual values and corresponding predictions, defined above
correct = actual == predictions
tp = numpy.sum(correct & actual)
tn = numpy.sum(correct & numpy.logical_not(actual))
fp = numpy.sum(numpy.logical_not(correct) & actual)
fn = numpy.sum(numpy.logical_not(correct) & numpy.logical_not(actual))

print("TP - True Positives: ", tp)
print("TN - True Negatives: ", tn)
print("FP - False positives: ", fp)
print("FN - False negatives: ", fn)

Podemos usar los valores y la matriz anteriores para ayudarnos a comprender otras métricas.

Cálculo de métricas
De aquí en adelante, analizaremos más de cerca cada una de las siguientes métricas, cómo se calculan y cómo pueden ayudar a explicar nuestro modelo actual.

Exactitud<br>
Sensibilidad/recuperación<br>
especificidad<br>
Precisión<br>
Tasa de falsos positivos<br>
Primero recordemos algunos términos útiles:<br>

TP = Verdaderos positivos: una etiqueta positiva se predice correctamente<br>
TN = Verdaderos negativos: una etiqueta negativa se predice correctamente<br>
FP = Falsos positivos: una etiqueta negativa se predice como positiva<br>
FN = Falsos negativos: una etiqueta positiva se predice como negativa<br>
Exactitud<br>
La precisión es el número de predicciones correctas dividido por el número total de predicciones:<br>

   accuracy = (TP+TN) / number of samples

It's possibly the most basic metric used but, as we've seen, it's not the most reliable when imbalanced datasets are used.

In code:

In [ ]:
# code for sensitivity/recall
sensitivity = recall = tp / (tp + fn)

# print result as a percentage
print(f"Model sensitivity/recall is {sensitivity:.2f}%")

especificidad

La especificidad expresa la fracción de etiquetas negativas predichas correctamente sobre el número total de muestras negativas existentes:

     especificidad = TN / (TN + FP)
     
Nos dice cómo, de todas las muestras realmente negativas, cuántas se predicen correctamente como negativas

Se puede calcular con el siguiente código:

In [ ]:
# Code for specificity
specificity = tn / (tn + fp)

# print result as a percentage
print(f"Model specificity is {specificity:.2f}%")

Precisión
La precisión expresa la proporción de muestras positivas pronosticadas correctamente sobre todas las predicciones positivas:

     precisión = TP / (TP + FP)
     
En otras palabras, indica cómo de todas las predicciones positivas, cuántas son etiquetas verdaderamente positivas.

Se puede calcular con el siguiente código:

In [ ]:
# Code for precision

precision = tp / (tp + fp)

# print result as a percentage
print(f"Model precision is {precision:.2f}%")

Tasa de falsos positivos
Tasa de falsos positivos o FPR, es el número de predicciones positivas incorrectas dividido por el número total de muestras negativas:

     tasa_falso_positivo = FP / (FP + TN)
     
De todos los negativos reales, ¿cuántos se clasificaron erróneamente como positivos?

En codigo:

In [ ]:
# Code for false positive rate
false_positive_rate = fp / (fp + tn)

# print result as a percentage
print(f"Model false positive rate is {false_positive_rate:.2f}%")


Tenga en cuenta que la suma de la especificidad y la tasa de falsos positivos siempre debe ser igual a 1.

Conclusión
Hay varias métricas diferentes que pueden ayudarnos a evaluar el rendimiento de un modelo, en el contexto de la calidad de sus predicciones.

La elección de las métricas más adecuadas, sin embargo, es principalmente una función de los datos y del problema que estamos tratando de resolver.

Resumen
Cubrimos los siguientes temas en esta unidad:

Cómo calcular las medidas muy básicas utilizadas en la evaluación de modelos de clasificación: TP, FP, TN, FN.<br>
Cómo usar las medidas anteriores para calcular métricas más significativas, como:<br>
Exactitud<br>
Sensibilidad/recuperación<br>
especificidad<br>
Precisión<br>
Tasa de falsos positivos<br>
Cómo la elección de las métricas depende del conjunto de datos y del problema que estamos tratando de resolver.